In [15]:
import os
import sys
import ctypes
import math
import numpy
import numpy as np
import h5py
import tiledb

In [16]:
# Total number of time steps
# num_time_steps = 257
num_time_steps = 4

# Shape of the array for each time step
array_shape = (1024, 1024, 1024)

# Iterate over the .npy files and write to the HDF5 dataset
for timestep in range(1,num_time_steps):
    # Load the .npy file
    file_path = f'/home2/aoyagir/isotropic1024coarse/pressure2_time_steps/t_{timestep:04}.npy'
    if os.path.exists(file_path):
        # print("iine")
        pass
    else:
        print("doesnt exists")

In [12]:
# Create a TileDB config
config = tiledb.Config()

# Set Minio-specific configuration options
 

config["vfs.s3.scheme"] = "http"
config["vfs.s3.region"] = ""
config["vfs.s3.endpoint_override"] = "172.20.2.253:9000"
config["vfs.s3.use_virtual_addressing"] = "false"
config["vfs.s3.aws_access_key_id"] = "189XB7837GTE0Kt1lv6b"
config["vfs.s3.aws_secret_access_key"] = "aQ3nGhtwk8Qod24f201KAMzShU5Y5VUgVOQL0XG5"

# Create contex
ctx = tiledb.Ctx(config)



In [20]:
array_name = f"array{num_time_steps}"
# array_name = f"array_{num_time_steps}lz4"
# array_name = f"array_{num_time_steps}zstd"
# array_name = f"array_{num_time_steps}gzip"


# array_uri = f"/home2/aoyagir/tiledb_data/{array_name}"
array_uri = f"s3://{array_name}"
print(array_name)

array_schema = tiledb.ArraySchema(
    domain=tiledb.Domain(
        tiledb.Dim(name="timestep",domain=(0, num_time_steps - 1), tile=1, dtype=np.uint64),
        tiledb.Dim(name="x", domain=(0, 1023), tile=256, dtype=np.uint64),
        tiledb.Dim(name="y", domain=(0, 1023), tile=256, dtype=np.uint64),
        tiledb.Dim(name="z", domain=(0, 1023), tile=256, dtype=np.uint64),
    ),
    attrs=[tiledb.Attr(name="data", dtype=np.float32)],
    # filters=tiledb.FilterList([tiledb.LZ4Filter()]),  # Enable LZ4 compression
    # filters=tiledb.FilterList([gzip_filter])
    # filters=tiledb.FilterList([zstd_filter])
    # filters=tiledb.FilterList([blosc_filter])
)

tiledb.Array.create(array_uri, array_schema,ctx=ctx)

array4


In [21]:
# Iterate over the .npy files and write to the TileDB dataset
for timestep in range(1, num_time_steps):
    file_path = f'/home2/aoyagir/isotropic1024coarse/pressure2_time_steps/t_{timestep:04}.npy'
    if os.path.exists(file_path):
        # Load the .npy file
        data_3d = np.load(file_path).astype(np.float32)

        # Open the TileDB array for writing
        with tiledb.DenseArray(array_uri, mode="w", ctx=ctx) as array:
            # Define the coordinates for this timestep
            coords = (timestep-1, slice(0, 1024), slice(0, 1024), slice(0, 1024))
            # Write the 3D data to the TileDB array
            array[coords] = data_3d
        print(f"timestep{timestep} is done")
    else:
        print(f"File doesn't exist for timestep {timestep}")

timestep1 is done
timestep2 is done
timestep3 is done


In [22]:
config = tiledb.Config()

# Set Minio-specific configuration options
 

config["vfs.s3.scheme"] = "http"
config["vfs.s3.region"] = ""
config["vfs.s3.endpoint_override"] = "172.20.2.253:9000"
config["vfs.s3.use_virtual_addressing"] = "false"
config["vfs.s3.aws_access_key_id"] = "189XB7837GTE0Kt1lv6b"
config["vfs.s3.aws_secret_access_key"] = "aQ3nGhtwk8Qod24f201KAMzShU5Y5VUgVOQL0XG5"

# Create contex
ctx = tiledb.Ctx(config)

In [23]:
with tiledb.open(array_uri, mode="r", ctx=ctx) as array:
    # Retrieve and print the array schema
    array_schema = array.schema
    print("Array Schema:")
    print(array_schema)

    # You can also access other information about the array, such as its domain and attributes
    domain = array_schema.domain


    print("Domain:", domain)




Array Schema:
ArraySchema(
  domain=Domain(*[
    Dim(name='timestep', domain=(0, 3), tile=1, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='x', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='y', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
    Dim(name='z', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
  ]),
  attrs=[
    Attr(name='data', dtype='float32', var=False, nullable=False, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  sparse=False,
)

Domain: Domain(Dim(name='timestep', domain=(0, 3), tile=1, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
       Dim(name='x', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1), ])),
       Dim(name='y', domain=(0, 1023), tile=256, dtype='uint64', filters=FilterList([ZstdFilter(level=-1),

In [27]:
with tiledb.DenseArray(array_uri, mode="r", ctx=ctx) as array:
    # Read the sliced data from the TileDB array using 'from' and 'to' expressions
    data = array[0, 0:256, 0:256, 0:256]
    print(data)

OrderedDict([('data', array([[[ 0.30191338,  0.2950729 ,  0.28597635, ..., -0.28747082,
         -0.29483193, -0.30216688],
        [ 0.30330813,  0.29460722,  0.2836517 , ..., -0.29126596,
         -0.29815337, -0.30518842],
        [ 0.2975547 ,  0.28577524,  0.27363893, ..., -0.29572615,
         -0.30240864, -0.3094731 ],
        ...,
        [-0.78851926, -0.79056543, -0.79237366, ...,  0.02012964,
          0.01824734,  0.01457602],
        [-0.78407156, -0.78569424, -0.7869962 , ...,  0.01964203,
          0.0169096 ,  0.01383047],
        [-0.779228  , -0.7805062 , -0.7815094 , ...,  0.02062516,
          0.01819261,  0.01639227]],

       [[ 0.29940894,  0.29601768,  0.29162994, ..., -0.28254396,
         -0.29037338, -0.29868338],
        [ 0.3002585 ,  0.29563743,  0.289917  , ..., -0.2869631 ,
         -0.2942592 , -0.30214715],
        [ 0.29535323,  0.2889055 ,  0.28194022, ..., -0.29204836,
         -0.29905045, -0.30672434],
        ...,
        [-0.8062171 , -0.8082939